# Read the important libraries

In [ ]:
!pip install transformers
# !pip install rouge

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 63.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 93.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.3 MB/s eta 0:00:00


In [ ]:
import json # for dealing with Json files
import pandas as pd # for dealing with dataset
import csv # used to wirte the data in CVS file

import nltk
nltk.download('stopwords') # download stopwords to remove them from your text data
from nltk.corpus import stopwords # used to remove stopwords from the dataset
nltk.download('punkt') # pre-trained tokenizer for tokenizing text into sentences.
from nltk.tokenize import word_tokenize # to split the text into list of words

import torch # to import cuda.is_available() method to run on GPU
from transformers import AutoTokenizer, pipeline

from rouge import Rouge # To import ROUGE metrics for evaluation

import joblib # To save model

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Convert Json data to CVS file

In [ ]:
# The pass to the Json dataset (old)
input_file = 'labeled_validation_dataset.jsonl'
# The pass to the CSV dataset (new)
output_file = 'labeled_validation_dataset.csv'

# Read from the Json file and write in the CSV file
with open(input_file, 'r', encoding='utf-8') as input, open(output_file, 'w', encoding='utf-8', newline='') as output:
    # Write the rows in csv
    writer = csv.writer(output)
    # Create column names
    writer.writerow(['paragraph', 'summary'])

    # Loop on the input file and copy his content into output file
    for line in input:
        # Load the lines from Json data
        data = json.loads(line)
        # Copy the data
        paragraph = data['paragraph']
        summary = data['summary']

        # Write the extracted data to the output file
        writer.writerow([paragraph, summary])


# 🔍 Step 1: Exploratory Data Analysis (EDA)

# Show the new dataset

In [ ]:
# Read the new dataset that we was created in the above
dataset = pd.read_csv('labeled_validation_dataset.csv')
# display the dataset
dataset.head()

,input_text,target_text
0,وتحت عنوان من الكارثة إلى التحدى يبدأ الكاتب ع...,يبدأ الكاتب عرض الكتاب الرابع تحت عنوان من الك...
1,ولم يعترف دبلوماسيو هاتين الدولتين بالعريضة ال...,دبلوماسيو الدولتين لم يعترفوا بالعريضة التي قا...
2,قامت ولاية حلب بعد اعلان الجنرال الفرنسي هنري ...,أعلن غورو الانتداب الفرنسي على سوريا لكي يعاقب...
3,دولة مصر العربيه هي ليست اي دوله وليست اي شعب ...,مصر هي أم البلاد، وقائدة العرب؛ فهي أرض بلاد ا...
4,السوريون يصرون على استقلال بلادهم : و مثلما رف...,الشعب السوري يصر على استقلال بلدهم من السيطرة ...


# Step 3: Load the Pretrained Transformer Model (mT5) on GPU

In [ ]:
# model_name = "arabert/arb-summarization-pegasus"
# model_name = "facebook/bart-large"
# model_name = "asafaya/bert-base-arabic-t5"

# define pretrained model on GPU
model_name= "eslamxm/mt5-base-arabic"
tokenizer = AutoTokenizer.from_pretrained(model_name)
device = "cuda" if torch.cuda.is_available() else "cpu"
# model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
# pass first parameter is 'summarization' for abstractive text summarization.
model = pipeline("summarization", model=model_name, tokenizer=model_name, device=device)


# Step 4: Tokenization and get Inference or hypotheses on GPU

In [ ]:
generated_summaries = []

for index, row in dataset.iterrows():
    input_text = row['input_text']
    # encoding step
    # input_tokens = tokenizer.encode(input_text, truncation=True, padding=True, return_tensors="pt").to(device)
    # # generation step
    # summary_ids = model.generate(input_tokens, num_beams=4, max_length=100, early_stopping=True)
    # # decoding step
    # summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    # we can replace 3 steps with one step pipeline
    summary = model(input_text, max_length=100, num_beams=4, early_stopping=True)[0]['summary_text']
    '''
    + Encoding:
    - Pipeline automatically handles the encoding of the input text using the specified tokenizer associated with the model.
    + Generation:
    - Handles the generation of the summary by specifying the 'max_length', 'num_beams', and 'early_stopping' parameters.
    + Decoding:
    - Extract index 0 which is generated summary from the pipeline's output, which is a dictionary.
    '''
    generated_summaries.append(summary)

Add the column generated summaries or hypotheses to the dataset

In [ ]:
dataset['generated_summary'] = generated_summaries

Print the updated dataset with generated summaries

In [ ]:
dataset.head()

,input_text,target_text,generated_summary
0,وتحت عنوان من الكارثة إلى التحدى يبدأ الكاتب ع...,يبدأ الكاتب عرض الكتاب الرابع تحت عنوان من الك...,ناقشت صحف بريطانية الأحداث التي جرت خلال الحرب...
1,ولم يعترف دبلوماسيو هاتين الدولتين بالعريضة ال...,دبلوماسيو الدولتين لم يعترفوا بالعريضة التي قا...,أثارت العريضة التي وقعها أعضاء مجلس الأمن الدو...
2,قامت ولاية حلب بعد اعلان الجنرال الفرنسي هنري ...,أعلن غورو الانتداب الفرنسي على سوريا لكي يعاقب...,كانت الولايات الثلاث المنفصلة في سوريا، وهي ال...
3,دولة مصر العربيه هي ليست اي دوله وليست اي شعب ...,مصر هي أم البلاد، وقائدة العرب؛ فهي أرض بلاد ا...,على مدى عقود، كانت مصر على مدى عقود، دولة عربي...
4,السوريون يصرون على استقلال بلادهم : و مثلما رف...,الشعب السوري يصر على استقلال بلدهم من السيطرة ...,أصبحت سوريا عاصمة لفرنسا، وهي ثاني دولة في الع...


# Step 5: Evaluation using ROUGE metrics
+ ROUGE: (Recall-Oriented Understudy for Gisting Evaluation)
+ used for evaluating the quality of text summarization outputs.

Initialize object from ROUGE metric

In [ ]:
rouge = Rouge()

+ Prepare the references and hypotheses:
  - column [summary] is references..
      - Convert the reference summaries from the dataset into a list
  - column [generated_summary] is hypotheses..
      - Convert the generated summaries into a list

In [ ]:
references = dataset['target_text'].tolist()
hypotheses = dataset['generated_summary'].tolist()

- Pass hypotheses and references for method get_scores to compute the ROUGE scores:
- avg= True >> compute average scores for all the scores.

In [ ]:
scores = rouge.get_scores(hypotheses, references, avg=True)

- ROUGE-1: measures the overlap of `unigrams (single words)` between the generated summary (hypotheses) and the reference summary.

- ROUGE-2: measures the overlap of `bigrams (pairs of consecutive words)` between the generated summary (hypotheses) and the reference summary.

- ROUGE-L: measures the `longest common subsequence (LCS)` between the generated summary (hypotheses) and the reference summary.

- Precision: ratio of the number of overlapping unigrams (ROUGE-1), bigrams (ROUGE-2) or LCS (ROUGE-L) between the generated and reference summaries to the number of unigrams or bigrams or LCS in the `generated summary`.

- Recall: ratio of the number of overlapping unigrams (ROUGE-1), bigrams (ROUGE-2) or LCS (ROUGE-L) between the generated and reference summaries to the number of unigrams or bigrams or LCS in the `reference summary`.

- ROUGE-1 Score: `harmonic mean` of precision and recall.

In [ ]:
print("ROUGE-1: ", scores['rouge-1'])
print("ROUGE-2: ", scores['rouge-2'])
print("ROUGE-L: ", scores['rouge-l'])
print("Precision: ", scores['rouge-1']['p'])
print("Recall: ", scores['rouge-1']['r'])
print("ROUGE-1 Score: ", scores['rouge-1']['f'])

ROUGE-1:  {'r': 0.06370570285296524, 'p': 0.263890547862241, 'f': 0.10072071605934921}
ROUGE-2:  {'r': 0.013874044445197744, 'p': 0.0686045140925455, 'f': 0.02265967889551928}
ROUGE-L:  {'r': 0.050551277095026506, 'p': 0.21398217612576342, 'f': 0.08022993746180616}
Precision:  0.263890547862241
Recall:  0.06370570285296524
ROUGE-1 Score:  0.10072071605934921


# Step 6: Save the model

In [ ]:
joblib.dump(model, r'AS_without_preprocessing.pkl')

['AS_without_preprocessing.pkl']

# Step 7: Deployment

In [ ]:
model = joblib.load('AS_without_preprocessing.pkl')

In [ ]:
input_text = "لم يعترف دبلوماسيو هاتين الدولتين بالعريضة التي وقعها الأعيان ، إذ اعتبروها محاولة فقط من سلطات الحماية لإجبار السلطان على قبول مخططاتها وذلك بالتوقيع على الظهائر . أما معارضة الدول العربية والآسيوية ، فإنها تمثلت في شنها لحملة شرسة عبر الصحافة والإذاعة ضد الإجراء الفرنسي . فما إن أعلن عن نبأ تنحية السلطان حتى أدان زعيم الحركة الوطنية علال الفاسي عبر إذاعة القاهرة في برنامج صوت العرب إبعاد السلطان هو وعائلته عن الوطن . وأما جامعة الدول العربية ، فقد عبرت عن تخوفاتها إزاء تطورات القضية المغربية وبدأت في الإلحاح على ضرورة استقلال المغرب . ومنذ 21 أغسطس 1953، أظهرت 15 دولة عربية آسيوية عضو في هيئة الأمم المتحدة انشغالها بمستقبل الاستقرار السياسي بشمال إفريقيا بعد عملية نفي محمد بن يوسف . وعلى مستوى العلاقات السياسية بين فرنسا وإسبانيا ، حدث الأمر نفسه . فحسب الوفق الذي أبرم بين البلدين يوم 27 نوفمبر 1912، فقد حدد أنه يتعهد البلدان بالتزام \" الاحترام إزاء الإمبراطورية الشريفة \"، في المنطقتين ، الخليفية التابعة للحماية الإسبانية ، والجنوبية ، الخاضعة للحماية الفرنسية . وقد استقبلت عملية تنصيب سلطان جديد على المغرب بإسبانيا بنوع من الاستياء ، إذ اعتبرته سلطات مدريد إعلانا لنوع من العداء ضدها . مما دفعها إلى تنظيم عملية توقيع عريضة مشابهة لتلك التي تمت بمراكش في المنطقة التي تخضع لسيطرتها . وتمكنت من جمع 430 توقيعا من باشوات وأعيان المنطقة الخليفية ركزت فيها على تثبيت فكرة \" تنحية السلطان الشرعي محمد بن يوسف ، نتيجة للدسائس التي حاكتها الإقامة العامة \" ورفض \" سلطة ابن عرفة .. والتي فرضتها فرنسا ضد إرادة الشعب المغربي \". ثم أعلنت العريضة \" أحقية السيادة التي يتمتع بها المهدي بن إسماعيل على المنطقة الخليفية \"، وذلك تحت إمرة سلطة فرانكو ."

In [ ]:
summary_text = model(input_text, max_length=100, num_beams=4, early_stopping=True)[0]['summary_text']

In [ ]:
print(summary_text)

أثارت العريضة التي وقعها أعضاء مجلس الأمن الوطني الفرنسي، التي وقعت في مدينة مدريد الإسبانية، جدلا واسعا عبر الصحافة والإذاعة حول مستقبل الاستقرار السياسي في شمال إفريقيا.


# Step 8: Submission

In [ ]:
import pandas as pd
from zipfile import ZipFile
from typing import Dict, List
import random
import os
from tempfile import TemporaryDirectory

def create_submission(output_file_path : str, submission_dictionary : Dict[int, str], base_keys : List[int]) -> None:
    """Function that validates the submission data types and schema and zip it to be ready from submission

    Parameters
    ----------
    output_file_path : str
        The locaiton and file name you want to save the zip file at, ex : "/home/user/submission_123.zip"
    submission_dictionary : dict[int, str]
        dictionary of int keys (example_id) and string values (summary)
    base_keys: list[int]
        list of keys of the original unlabeled validation set


    Returns
    -------
    None
    """
    #assertions
    assert all(isinstance(i, int) for i in submission_dictionary.keys()), "Make sure example_ids elements (key of submission_dictionary) are of type int"
    assert all(isinstance(i, str) for i in submission_dictionary.values()), "Make sure summary elements (value of submission_dictionary) are of type str"
    assert all(isinstance(i, int) for i in base_keys), "Make sure base_keys elements is of type int"

    diff_sub = set(submission_dictionary.keys()) - set(base_keys)
    diff_base = set(base_keys) - set(submission_dictionary.keys())

    assert len(diff_sub) == 0, f"Keys {diff_sub} is in submission but not in base_keys"
    assert len(diff_base) == 0, f"Keys {diff_base} is in base_keys but not in submission"

    #saving
    final_submission = pd.DataFrame(submission_dictionary.items(), columns=['example_id', 'summary'])

    if final_submission.example_id.dtype != 'int64' :
        final_submission.example_id = final_submission.example_id.astype(int)

    assert len(final_submission[final_submission.summary.isna()]) == 0, f"summaries with the example_id = {final_submission[final_submission.summary.isna()].example_id.values.tolist()} is NaN"
    assert len(final_submission[final_submission.example_id.isna()]) == 0, f"example_ids with the following index = {final_submission[final_submission.example_id.isna()].index.tolist()} is NaN"

    with TemporaryDirectory(dir=".") as tmpdirname:
        os.chdir(tmpdirname)
        jsonl_name = "predictions.jsonl"
        final_submission.to_json(jsonl_name, lines=True, orient='records', force_ascii=False)
        with ZipFile(output_file_path, "w") as zip_file:
            zip_file.write(filename = jsonl_name)
            print(f"Submission of {jsonl_name} as .zip saved at {output_file_path}")
        os.chdir("..")

In [ ]:
val_data = pd.read_json("validation_data.jsonl", lines=True)

In [ ]:
val_data.head()

In [ ]:
val_data['summary'] = val_data.paragraph.str.split("\n").apply(lambda x: "\n".join(random.choices(population = x, k=2)))
val_data